<a href="https://colab.research.google.com/github/OneFineStarstuff/OneFineStardust/blob/main/Hyperparameter_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install keras_tuner

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from keras_tuner import RandomSearch

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize the images to values between 0 and 1
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# Reshape data to fit model input shape (28, 28, 1)
x_train_cnn = x_train.reshape((60000, 28, 28, 1))
x_test_cnn = x_test.reshape((10000, 28, 28, 1))

# Define the model building function
def build_model(hp):
    model = models.Sequential()
    model.add(layers.Conv2D(hp.Int('conv_units', min_value=32, max_value=128, step=32), (3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(hp.Int('dense_units', min_value=64, max_value=256, step=64), activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Initialize the tuner
tuner = RandomSearch(build_model,
                     objective='val_accuracy',
                     max_trials=5)

# Perform the search
tuner.search(x_train_cnn, y_train, epochs=5, validation_split=0.2)

# Print the best hyperparameters found
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f'Best hyperparameters: Conv units: {best_hps.get("conv_units")}, Dense units: {best_hps.get("dense_units")}')